<a href="https://colab.research.google.com/github/fedhere/MLPNS_FBianco/blob/main/KNN/KNN_wine_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# get the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

In [ ]:
cd /content/drive/MyDrive/

In [ ]:
!mkdir MLPNS2023

In [ ]:
cd MLPNS2023

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.names

# ingest the data

In [ ]:
import pandas as pd

In [ ]:
wines = pd.read_csv("wine.data", header=None)
wines

In [ ]:
columns = {0:"quality",
           1:"Alcohol",2:"Malic acid",3:"Ash",4:"Alcalinity of ash",5:"Magnesium",
	        6:"Total phenols",7:"Flavanoids",8:"Nonflavanoid phenols",9:"Proanthocyanins",
          10:"Color intensity",11:"Hue",12:"OD280/OD315 of diluted wines",13:"Proline"}

wines.rename(columns=columns, inplace=True)
wines

# preprocessing

In [ ]:
from sklearn import preprocessing

In [ ]:
X = preprocessing.scale(wines.iloc[:,1:])

In [ ]:
X.mean(axis=0).shape

# data exploration

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, wines["quality"], random_state=302)

In [ ]:
import sklearn.manifold as man
tsne_wines = man.TSNE(n_components=2, perplexity=30.0, early_exaggeration=90.0,
                      random_state=302).fit(X)
wine_proj = tsne_wines.fit_transform(X)
wine_proj_test = tsne_wines.fit_transform(X_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import DecisionBoundaryDisplay
_, ax = plt.subplots()


ax.scatter(wine_proj.T[0], wine_proj.T[1], s=10, c = wines["quality"]/3);
ax.set_title("true labels")

# modeling

In [ ]:
import sklearn.neighbors as nb

In [ ]:
clf = nb.KNeighborsClassifier()
clf.fit(X_train, y_train)

In [ ]:
pred = clf.predict(X_test)
pred

In [ ]:
y_score = clf.predict_proba(X_test)
y_score

In [ ]:


_, ax = plt.subplots()
ax.scatter(wine_proj_test.T[0], wine_proj_test.T[1], s=40, c = y_test/3);
ax.set_title("true labels")

_, ax = plt.subplots(1)

ax.scatter(wine_proj_test.T[0], wine_proj_test.T[1], s=40, c = pred/3);
ax.set_title("predicted labels");


# model evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, pred);
disp = ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test, cmap="bone")

In [ ]:
disp = ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test, cmap="bone", normalize="true")

In [ ]:
y_pred

In [ ]:
from sklearn import metrics

precision, recall = metrics.precision_score(y_test, pred, average='macro'), \
                    metrics.recall_score(y_test, pred, average='macro')


precision, recall

In [ ]:
from sklearn.metrics import RocCurveDisplay
import numpy as np
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_test)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

label_binarizer.transform([0])



In [ ]:
label_binarizer.transform(y_test)

In [ ]:

class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
y_score = clf.fit(X, wines["quality"]).predict_proba(X_test)
y_score

In [ ]:
for i in range(1, 4):
  class_of_interest = i
  class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
 

  RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    color="darkorange",
  )
  plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
  plt.axis("square")
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("One-vs-Rest ROC curves:\n {} vs other".format(class_of_interest))
  plt.legend()
  plt.show()


In [ ]:
y_score

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=2)
auc = roc_auc_score(label_binarizer.transform(y_test), y_score, multi_class="ovr")
auc